In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np

# Load the combined narrative data
data_dir = '/content/drive/MyDrive/Colab Notebooks/Berkeley/266/FinalProject/Data/'
original_data = os.path.join(data_dir, 'sentences_with_labels.csv')

df = pd.read_csv(original_data)

# Fix Nan to ""
df = df.fillna("")

display(df.head())

print(df.shape)

,sentence1,sentence2,sentence3,sentence4,sentence5,mistral_sentence5,llama_sentence5,label
0,Frank had been drinking beer.,"He got a call from his girlfriend, asking wher...",Frank suddenly realized he had a date that night.,"Since Frank was already a bit drunk, he could ...",Frank spent the rest of the night drinking mor...,Frank called an Uber and was able to make it t...,"He called a taxi to pick him up, but it took a...",0
1,Dave was in the Bahamas on vacation.,He decided to go snorkeling on his second day.,"While snorkeling, he saw a cave up ahead.","He went into the cave, and he was terrified wh...","Dave swam away as fast as he could, but the sh...","Dave quickly swam back to the surface, scared ...",He swam back to the surface and quickly paddle...,1
2,Sunny enjoyed going to the beach.,"As she stepped out of her car, she realized sh...",It was quite sunny and she forgot her sunglasses.,Sunny got back into her car and heading toward...,Sunny found some sunglasses and headed back to...,Sunny bought a new pair of sunglasses at the m...,She went to the mall to buy a new pair of sung...,0
3,Sally was happy when her widowed mom found a n...,She discovered her siblings didn't feel the same.,Sally flew to visit her mom and her mom's new ...,"Although her mom was obviously in love, he was...",Sally went home and wondered about her parents...,"Although Sally struggled at first, she eventua...","Her mom's new husband seemed kind, but Sally n...",1
4,Dan hit his golf ball and watched it go.,The ball bounced on the grass and into the san...,Dan pretended that his ball actually landed on...,His friends were not paying attention so they ...,Dan snuck a ball on the green and made his put...,Dan's friends eventually noticed that his ball...,"His friends were impressed by his fake shot, b...",0


(40000, 8)


In [3]:
!pip install transformers datasets -q

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch

In [5]:
# Load your dataset
df = df[['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5',
         'mistral_sentence5', 'llama_sentence5', 'label']]

# Combine sentences into single inputs
def prepare_inputs(row):
    """
    Combine the context sentences with each possible ending into a format suitable for BERT.
    """
    mistral_input = f"{row['sentence1']} {row['sentence2']} {row['sentence3']} {row['sentence4']} {row['mistral_sentence5']}"
    llama_input = f"{row['sentence1']} {row['sentence2']} {row['sentence3']} {row['sentence4']} {row['llama_sentence5']}"
    return mistral_input, llama_input

df['mistral_input'], df['llama_input'] = zip(*df.apply(prepare_inputs, axis=1))

# Split dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Tokenizer and preprocessing
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples['mistral_input'],
        examples['llama_input'],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors='pt'
    )

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['mistral_input', 'llama_input', 'label']])
val_dataset = Dataset.from_pandas(val_df[['mistral_input', 'llama_input', 'label']])

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(['mistral_input', 'llama_input'])
val_dataset = val_dataset.remove_columns(['mistral_input', 'llama_input'])

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [8]:
# Model
def get_model():
  return BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
#model = get_model()

In [9]:
def get_trainer(
    model,
    train_dataset,
    val_dataset,
    tokenizer,
    learning_rate=2e-5,
    batch_size=8,
    weight_decay=0.01,
    num_train_epochs=5
):
    training_args = TrainingArguments(
        output_dir='./results',
        eval_strategy='epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        save_strategy='epoch',
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        push_to_hub=False,
        report_to=["none"],  # Disable W&B logging
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = torch.argmax(torch.tensor(logits), dim=-1)
        accuracy = (predictions == labels).float().mean().item()
        return {'accuracy': accuracy}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    return trainer


In [10]:
results = {'LearningRate': [], 'BatchSize': [], 'Loss': [], 'Accuracy': []}
learning_rate = [1e-5, 2e-5]
batch_size = [8, 16, 32]

for lr in learning_rate:
    for bs in batch_size:
        print(f"Testing: LR={lr}, BS={bs}")

        # Initialize a new model for each configuration
        model = get_model()

        # Create the trainer with the specified hyperparameters
        trainer = get_trainer(
            model=model,
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            tokenizer=tokenizer,
            learning_rate=lr,
            batch_size=bs
        )

        # Train the model
        trainer.train()

        # Evaluate the model
        eval_results = trainer.evaluate()

        # Log results
        print(f"Validation Results for LR={lr}, BS={bs}: {eval_results}")
        results['LearningRate'].append(lr)
        results['BatchSize'].append(bs)
        results['Loss'].append(eval_results.get('eval_loss', None))
        results['Accuracy'].append(eval_results.get('eval_accuracy', None))


Testing: LR=1e-05, BS=8


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-1c43f7d53aa3>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.664100,0.645657,0.616375
2,0.511900,0.643939,0.632625
3,0.496700,0.715385,0.622875
4,0.416900,0.892362,0.625000
5,0.271100,1.186266,0.622875


Validation Results for LR=1e-05, BS=8: {'eval_loss': 0.6439393162727356, 'eval_accuracy': 0.6326249837875366, 'eval_runtime': 23.1353, 'eval_samples_per_second': 345.792, 'eval_steps_per_second': 43.224, 'epoch': 5.0}
Testing: LR=1e-05, BS=16


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.631400,0.668508,0.610750
2,0.592600,0.646293,0.621500
3,0.474900,0.748215,0.621625
4,0.398000,0.910495,0.621750
5,0.234400,1.256693,0.620750


Validation Results for LR=2e-05, BS=16: {'eval_loss': 0.910495400428772, 'eval_accuracy': 0.621749997138977, 'eval_runtime': 20.5371, 'eval_samples_per_second': 389.538, 'eval_steps_per_second': 24.346, 'epoch': 5.0}
Testing: LR=2e-05, BS=32


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-1c43f7d53aa3>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.678300,0.671556,0.577000
2,0.623900,0.639952,0.615500
3,0.542800,0.682718,0.623375
4,0.493900,0.784537,0.621750
5,0.303300,0.903538,0.619750


Validation Results for LR=2e-05, BS=32: {'eval_loss': 0.6827182769775391, 'eval_accuracy': 0.6233749985694885, 'eval_runtime': 18.6102, 'eval_samples_per_second': 429.871, 'eval_steps_per_second': 13.433, 'epoch': 5.0}


In [11]:
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(data_dir, 'results.csv'), index=False)
display(results_df)

,LearningRate,BatchSize,Loss,Accuracy
0,0.00001,8,0.643939,0.632625
1,0.00001,16,0.635025,0.622750
2,0.00001,32,0.642510,0.626500
3,0.00002,8,0.642044,0.627375
4,0.00002,16,0.910495,0.621750
5,0.00002,32,0.682718,0.623375


In [12]:
from google.colab import runtime
runtime.unassign()

In [ ]:
# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Save the model
model.save_pretrained('./best_model')
tokenizer.save_pretrained('./best_model')

Epoch,Training Loss,Validation Loss,Accuracy
1,0.656900,0.657578,0.605250
2,0.644100,0.651822,0.616250
3,0.451500,0.798858,0.612375
4,0.528100,1.205326,0.622750
5,0.238700,1.933847,0.618375


('./best_model/tokenizer_config.json',
 './best_model/special_tokens_map.json',
 './best_model/vocab.txt',
 './best_model/added_tokens.json')

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print the evaluation results explicitly
print(f"Validation Results: {eval_results}")

Validation Results: {'eval_loss': 1.2053260803222656, 'eval_accuracy': 0.6227499842643738, 'eval_runtime': 87.6227, 'eval_samples_per_second': 91.301, 'eval_steps_per_second': 11.413, 'epoch': 5.0}


In [ ]:
from sklearn.metrics import classification_report

predictions = trainer.predict(val_dataset)

logits = predictions.predictions
predicted_classes = np.argmax(logits, axis=1)
true_classes = predictions.label_ids

report = classification_report(true_classes, predicted_classes, target_names=['Mistral', 'Llama'])
print(report)

              precision    recall  f1-score   support

     Mistral       0.63      0.65      0.64      4152
       Llama       0.61      0.59      0.60      3848

    accuracy                           0.62      8000
   macro avg       0.62      0.62      0.62      8000
weighted avg       0.62      0.62      0.62      8000



In [ ]:
## Save the model and tokenizer to Hugging Face Hub
model.push_to_hub(f"MatthewFrank/w266FinalProject")
tokenizer.push_to_hub(f"MatthewFrank/w266FinalProject")

In [ ]:
from google.colab import runtime
runtime.unassign()